In [86]:
# import the data
import pandas as pd
import numpy as np

path = 'assetclass_data_monthly_2009.xlsx'
df = pd.read_excel(path)
df.set_index('Dates',inplace=True)

In [87]:
# clean the data
df['rf'] = df['Cash'].fillna(0.01/12) # to fill the empty values

risky_assets = df.columns[0:11] # columns of indicies
df_tilde = df.loc[:,risky_assets]
df_tilde.subtract(df['rf'], axis = 0)

,Domestic Equity,Foreign Equity,Emerging Markets,Private Equity,Absolute Return,High Yield,Commodities,Real Estate,Domestic Bonds,Foreign Bonds,Inflation-Indexed
Dates,,,,,,,,,,,
2009-03-31,0.082349,0.082774,0.167482,0.152342,-0.012250,0.018166,0.043914,0.034089,0.031970,0.046377,0.057925
2009-04-30,0.098793,0.114638,0.155007,0.229647,0.022329,0.137877,-0.009741,0.295595,-0.028010,0.008439,-0.018521
2009-05-29,0.058921,0.132384,0.159879,0.054357,0.028333,0.028963,0.197137,0.023195,-0.020293,0.054136,0.020491
2009-06-30,-0.001272,-0.014908,-0.023120,0.044847,-0.004035,0.032761,0.005095,-0.025683,-0.006119,0.004552,0.001410
2009-07-31,0.074632,0.100439,0.110174,0.143274,0.015351,0.069189,0.004465,0.105825,0.008340,0.031310,0.000910
...,...,...,...,...,...,...,...,...,...,...,...
2019-05-31,-0.066198,-0.052755,-0.075726,-0.056143,-0.015476,-0.021744,-0.084561,-0.003921,0.028056,0.009023,0.015095
2019-06-28,0.067229,0.056738,0.059691,0.054962,0.015879,0.029173,0.041941,0.010348,0.009794,0.031919,0.005117
2019-07-31,0.013501,-0.021091,-0.028182,-0.004970,-0.002933,-0.000054,-0.003736,0.022205,-0.001231,-0.014542,0.001540


### For Problem 1
(a) Calculate and display the mean and volatility of each asset's excess return. (Recall we use
volatility to refer to standard deviation.)

(b) Which assets have the best and worst Sharpe ratios?

In [88]:
df_tilde_annual = df_tilde*12
mu_tilde = df_tilde_annual.mean() # mu_tilde is the annual mean
sigma = df_tilde_annual.std()/np.sqrt(12)

tabel_1 = pd.DataFrame({'Mean':mu_tilde, 'Volatility':sigma, 'Sharpe':mu_tilde/sigma})
tabel_1

,Mean,Volatility,Sharpe
Domestic Equity,0.161179,0.129421,1.245379
Foreign Equity,0.102351,0.157513,0.649794
Emerging Markets,0.101234,0.201679,0.501956
Private Equity,0.168527,0.199450,0.844960
Absolute Return,0.028065,0.044154,0.635627
High Yield,0.093070,0.082830,1.123624
Commodities,-0.015270,0.190977,-0.079957
Real Estate,0.179612,0.175007,1.026310
Domestic Bonds,0.041959,0.058571,0.716379
Foreign Bonds,0.030151,0.077190,0.390615


### Problem 2: The MV Frontier
(a) Compute and display the weights of the tangency portfolios: $w^{tan}$.

(b) Compute the mean, volatility, and Sharpe ratio for the tangency portfolio corresponding to
$w^{tan}$.

* The MV Frontier:
$
\omega^t = (\frac{1}{1'\Sigma^{-1}\mu})\Sigma^{-1}\mu
$
* Where, $\Sigma$ is the **matrix of covariance of asset returns**, then we convert it
* Where, $\mu$ is the N*1 vector of mean returns

In [89]:
# Compute and display the tangency portfolios

# Remember to use the annualized data as the diagram in this function!!!

def compute_tangency(df_tilde_annual,diagonalize_Sigma=False):
    Sigma = df_tilde_annual.cov() # df_tilde is all risky assets

    # N is the number of risky_assets
    N = Sigma.shape[0] # the column number, big N
    Sigma_adj = Sigma.copy()

    if diagonalize_Sigma:
        Sigma_adj.loc[:,:] = np.diag(np.diag(Sigma_adj))

    mu_tilde = df_tilde_annual.mean() # will return the mean value for each column prices(each asset)
    Sigma_inv = np.linalg.inv(Sigma_adj) # the sigma inverse matrix
    weights = Sigma_inv @ mu_tilde / (np.ones(N) @ Sigma_inv @ mu_tilde)

    omega_tangency = pd.Series(weights,index=mu_tilde.index)
    return omega_tangency, mu_tilde, Sigma

omega_tangency, mu_tilde, Sigma = compute_tangency(df_tilde_annual)
omega_tangency

Domestic Equity      0.940305
Foreign Equity      -0.037456
Emerging Markets    -0.137785
Private Equity      -0.147648
Absolute Return     -0.879594
High Yield           0.699706
Commodities         -0.104370
Real Estate         -0.192864
Domestic Bonds       0.677625
Foreign Bonds       -0.030999
Inflation-Indexed    0.213079
dtype: float64

* To compute the mean, volatility and the sharpe ratio for the tangency

In [90]:
# Mean
# Now we have the weights of each portfolio, we just need to allocate the weight to each asset
mean = mu_tilde @ omega_tangency

# Volatility
volatility = np.sqrt(omega_tangency @ Sigma @ omega_tangency)/np.sqrt(12)

# Sharpe ratio
sharpe_ratio = mean/volatility

print("Mean: ", mean, 'Volatility: ', volatility, 'Sharpe Ratio: ',sharpe_ratio)

Mean:  0.15244823468891544 Volatility:  0.06257208228691906 Sharpe Ratio:  2.4363618584703124


### The Allocation
(a) Compute and display the weights of MV portfolios with target returns of $\mu^p$ = .0067

(b) What is the mean, volatility, and Sharpe ratio for $w^p$?

(c) Discuss the allocation. In which assets is the portfolio most long? And short?

(d) Does this line up with which assets have the strongest Sharpe ratios?

#### The allocation of the MV, given the expected target returns $\mu^p$
$
 \tilde\delta = (\frac{1'\Sigma^{-1}\tilde\mu}{{\tilde\mu}'\Sigma^{-1}\tilde\mu})\tilde\mu^p
$

In [91]:
def target_mv_portfolio(df_tilde, target_returns=0.0067, diagnolize_sigma = False
                        ):
    # compute MV optimal portfolio, given the target returns and set of excessive returns
    omega_tangency, mu_tilde, Sigma = compute_tangency(df_tilde)
    # to use the function created above
    Sigma_adj = Sigma.copy()
    N = Sigma_adj.shape[0]

    if diagnolize_sigma == True:
        Sigma_adj.loc[:,:] = np.diag(np.diag(Sigma_adj))

    Sigma_inv = np.linalg.inv(Sigma_adj)

    delta_tilde = (np.ones(N) @ Sigma_inv @ mu_tilde)/(mu_tilde @ Sigma_inv @ mu_tilde) * target_returns

    allocated_omega = omega_tangency * delta_tilde

    return allocated_omega

allocated_omega = target_mv_portfolio(df_tilde_annual,target_returns=0.0067*12)
print(allocated_omega)
# Now it returns the target weights
#TODO: note here if using @ to get product of matrices, the invert or not does not matter

Domestic Equity      0.495909
Foreign Equity      -0.019754
Emerging Markets    -0.072667
Private Equity      -0.077868
Absolute Return     -0.463891
High Yield           0.369020
Commodities         -0.055044
Real Estate         -0.101715
Domestic Bonds       0.357374
Foreign Bonds       -0.016349
Inflation-Indexed    0.112376
dtype: float64


In [92]:
# The mean, volatility and Sharpe Ratio for new portfolios

# Mean
mean_allocated = mu_tilde @ allocated_omega

# Volatility
vol_allocated = np.sqrt(allocated_omega @ Sigma @ allocated_omega)/np.sqrt(12)

# Sharpe Ratio
sharpe_ratio_allocated = mean_allocated / vol_allocated

print('Mean: ', mean_allocated, "Volatility: ", vol_allocated, 'Sharpe Ratio: ', sharpe_ratio_allocated)

Mean:  0.0804 Volatility:  0.03300002408118461 Sharpe Ratio:  2.4363618584703124


In [93]:
print('TO sort the weighted allocations')
allocated_omega.sort_values(ascending=False)

# then to get the most extreme long and the most extreme short

TO sort the weighted allocations


Domestic Equity      0.495909
High Yield           0.369020
Domestic Bonds       0.357374
Inflation-Indexed    0.112376
Foreign Bonds       -0.016349
Foreign Equity      -0.019754
Commodities         -0.055044
Emerging Markets    -0.072667
Private Equity      -0.077868
Real Estate         -0.101715
Absolute Return     -0.463891
dtype: float64

* The longest position is in Domestic Equity, which does have the highest SR.
* But the shortest positions are Absolute return and Foreign Equities which do not have the lowest SRs.
* This is a reminder that **MV optimization will put strong weights on some securities due to their covariances, not due to their means, vols, or Sharpe Ratios.**


In [94]:
# for comparison purpose only
compare = pd.DataFrame({'Allocated Weights':allocated_omega, 'Sharpe Ratio':tabel_1['Sharpe']},index=allocated_omega.index)
compare
# for answering the question 4, the lowest allocated weights do not have necessarily the lowest sharpe ratios

,Allocated Weights,Sharpe Ratio
Domestic Equity,0.495909,1.245379
Foreign Equity,-0.019754,0.649794
Emerging Markets,-0.072667,0.501956
Private Equity,-0.077868,0.844960
Absolute Return,-0.463891,0.635627
High Yield,0.369020,1.123624
Commodities,-0.055044,-0.079957
Real Estate,-0.101715,1.026310
Domestic Bonds,0.357374,0.716379
Foreign Bonds,-0.016349,0.390615


### Problem 4 Long Short Positions
* (a) Consider an allocation between only domestic and foreign equities. (Drop all other return
columns and recompute $w^p$ for $\mu^p$ = .0067.)
* (b) What is causing the extreme long-short position?
* (c) Make an adjustment to $\mu^{Foreign Equities}$ of +0.001, (+0.012 annualized.) Recompute $w^p$ for
$\mu^p$ = :0067 for these two assets.
How does the allocation among the two assets change?
* (d) What does this say about the statistical precision of the MV solutions?


In [95]:
# Consider the allocation only between domestic and foreign equities
equities = ['Domestic Equity','Foreign Equity']
df_equities = df_tilde_annual[equities].copy()

# Now compute the MV given the target returns
omega_tangency_equities_target = target_mv_portfolio(df_equities,target_returns=0.0067)
print(omega_tangency_equities_target)

Domestic Equity    0.062378
Foreign Equity    -0.032769
dtype: float64


* for question a: the domestic equity should be longed and foreign equity should be shorted.

 * The MV optimizer wants to fit the in-sample correlation too tightly by creating a strong long-short position which will net out risk and leave us long the higher mean-return security.
 * But statistical significance of the difference in the mean returns is weak. Out of sample this could easily go against us. Or the correlations may change and the long-short hedge worsens.

In [96]:
# Make an adjustment to µ foreign equities of +0.001, (+0.012 annualized.) Recompute wp for µ p = .0067 for these two assets.
# How does the allocation among the two assets change?
# to add returns to foreign equities
df_equities['Foreign Equity']+0.012
target = target_mv_portfolio(df_equities,target_returns=0.0067*12)
print(target)

Domestic Equity    0.748533
Foreign Equity    -0.393231
dtype: float64


* Note that when we adjust the mean returns by much less than our standard error, (just .001!) we got very similar allocation.
* Generally, we won't see this, and we end up having huge change in allocation with slight in asset return as MV estimation is too sensitive to in-sample noise (also mentioned in part b). In this case, we got lucky for this set of data.

### 5. Robustness

(a) Recalculate the full allocation, again with the unadjusted $\mu^{foreign equities}$ and again for $\mu^p$ =
0.0067. This time, make one change: in building $w^{tan}$, do not use $\Sigma$ as given in the formulas
in the lecture. Rather, use a diaganolized $\Sigma^D$, which zeroes out all non-diagonal elements
of the full covariance matrix.
How does the allocation look now?

(b) What does this suggest about the sensitivity of the solution to estimated means and estimated
covariances?

(c) HMC deals with this sensitivity by using explicit constraints on the allocation vector. Conceptually,
what are the pros/cons of doing that versus modifying the formula with $\Sigma^D$?

In [97]:
target_inv = target_mv_portfolio(df_tilde_annual,target_returns=0.0067*12, diagnolize_sigma=True)
print(target_inv.sort_values(ascending=False))
total_fraction = target_inv.sum()
print(total_fraction)
# The risk asset should account for 53%

Domestic Equity      0.950424
High Yield           0.707237
Domestic Bonds       0.684918
Inflation-Indexed    0.215372
Foreign Bonds       -0.031333
Foreign Equity      -0.037859
Commodities         -0.105493
Emerging Markets    -0.139268
Private Equity      -0.149237
Real Estate         -0.194940
Absolute Return     -0.889060
dtype: float64
1.0107621588841378


In [98]:
# The mean, volatility and Sharpe Ratio for the new portfolio
mean = mu_tilde @ target_inv
volatility = np.sqrt(target_inv @ Sigma @ target_inv)/np.sqrt(12)
Sharpe_ratio = mean / volatility
print('Mean: ', mean, "Volatility: ", volatility, "Sharpe Ratio: ", Sharpe_ratio)

Mean:  0.15408890681224394 Volatility:  0.06324549297820223 Sharpe Ratio:  2.4363618584703133


In [99]:
compare = pd.DataFrame({'Allocated Weights':allocated_omega,'Diagnolized Weights':target_inv})
print(compare)


                   Allocated Weights  Diagnolized Weights
Domestic Equity             0.495909             0.950424
Foreign Equity             -0.019754            -0.037859
Emerging Markets           -0.072667            -0.139268
Private Equity             -0.077868            -0.149237
Absolute Return            -0.463891            -0.889060
High Yield                  0.369020             0.707237
Commodities                -0.055044            -0.105493
Real Estate                -0.101715            -0.194940
Domestic Bonds              0.357374             0.684918
Foreign Bonds              -0.016349            -0.031333
Inflation-Indexed           0.112376             0.215372


In [104]:
## Out-of-sample performance
ofs_df = df_tilde_annual.loc[:'2016',:]
ofs_weights = target_mv_portfolio(ofs_df,target_returns= 0.0067)
ofs_weights

Domestic Equity      0.038818
Foreign Equity      -0.003337
Emerging Markets    -0.006627
Private Equity      -0.005208
Absolute Return     -0.026978
High Yield           0.026221
Commodities         -0.003070
Real Estate         -0.006982
Domestic Bonds       0.024405
Foreign Bonds       -0.005385
Inflation-Indexed    0.012536
dtype: float64

In [107]:
# mean
ofs_mean = mu_tilde @ ofs_weights # 矩阵相乘得到具体的数值

# volatility
ofs_volatility = np.sqrt(ofs_weights @ Sigma @ ofs_weights)/np.sqrt(12)

# sharpe ratio
ofs_sharpe_ratio = ofs_mean / ofs_volatility

print('ofs_mean:', ofs_mean, 'ofs_volatility:', ofs_volatility, 'ofs_sharpe_ratio:', ofs_sharpe_ratio)

ofs_mean: 0.0062142308465465085 ofs_volatility: 0.002586775095801821 ofs_sharpe_ratio: 2.4023081313222123


In [113]:
# re-do it from 2017 to 2019
df_ofs_2 = df_tilde_annual.loc[:'2017',:]
ofs_weights_2 = target_mv_portfolio(df_ofs_2,target_returns=0.0067)
ofs_weights_2

compare = pd.DataFrame({'In-Sample Weights':allocated_omega,'Out-of-Sample Weights 1':ofs_weights,'Out-of-Sample Weights 2':ofs_weights_2})

print(compare)

                   In-Sample Weights  Out-of-Sample Weights 1  \
Domestic Equity             0.495909                 0.038818   
Foreign Equity             -0.019754                -0.003337   
Emerging Markets           -0.072667                -0.006627   
Private Equity             -0.077868                -0.005208   
Absolute Return            -0.463891                -0.026978   
High Yield                  0.369020                 0.026221   
Commodities                -0.055044                -0.003070   
Real Estate                -0.101715                -0.006982   
Domestic Bonds              0.357374                 0.024405   
Foreign Bonds              -0.016349                -0.005385   
Inflation-Indexed           0.112376                 0.012536   

                   Out-of-Sample Weights 2  
Domestic Equity                   0.041626  
Foreign Equity                   -0.002534  
Emerging Markets                 -0.004498  
Private Equity                   -0.005

In [117]:
# mean
ofs_mean_2 = mu_tilde @ ofs_weights_2 # 矩阵相乘得到具体的数值

# volatility
ofs_volatility_2 = np.sqrt(ofs_weights_2 @ Sigma @ ofs_weights_2)/np.sqrt(12)

# sharpe ratio
ofs_sharpe_ratio_2 = ofs_mean_2 / ofs_volatility_2

print('ofs_mean:', ofs_mean_2, 'ofs_volatility:', ofs_volatility_2, 'ofs_sharpe_ratio:', ofs_sharpe_ratio_2)

# TODO: remember this method to create a comparing table
means = [mean_allocated,ofs_mean,ofs_mean_2]
volatilities = [vol_allocated,ofs_volatility,ofs_volatility_2]
sharpe_ratios = [sharpe_ratio_allocated,ofs_sharpe_ratio,ofs_sharpe_ratio_2]
index_comapre = ['In-sample MV','Out-of-sample_1 MV','Out-of-sample_2 MV']
comparing_parameters = pd.DataFrame({'Means':means, 'Volatilities':volatilities, 'Sharpe Ratios':sharpe_ratios},index=index_comapre)
print(comparing_parameters)

ofs_mean: 0.006553059645548 ofs_volatility: 0.0027318882903655988 ofs_sharpe_ratio: 2.3987289921986625
                       Means  Volatilities  Sharpe Ratios
In-sample MV        0.080400      0.033000       2.436362
Out-of-sample_1 MV  0.006214      0.002587       2.402308
Out-of-sample_2 MV  0.006553      0.002732       2.398729
